<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Chiangmai-Geography" data-toc-modified-id="Chiangmai-Geography-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Chiangmai Geography</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports 

In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *

In [15]:
b_folder='../data/pm25/'
a4th_folder ='../data/air4thai_hourly/'
a4th_folder_p ='../data/a4th_hourly/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [171]:
# berekely data
b_data, city_info = read_b_data(b_folder+'Chiang_Mai.txt')
b_data = b_data.set_index('datetime')
city_info

{'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

In [19]:
# load stations information for air4 Thai
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        cm_station_ids.append(stations['stationID'])
        
print(cm_station_ids)

['35t', '36t', 'm9', 'o10', 'o20', 'o22']


In [18]:
# combine new and old air4Thai data 
gas_cols = ['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']
for station_id in cm_station_ids:
    try: 
        old_data = pd.read_csv(aqm_folder + 'process/'+station_id + '.csv')
    except:
        old_data = pd.DataFrame()
    else:
        old_data['datetime'] = pd.to_datetime(old_data['datetime'])
        old_data = old_data.set_index('datetime')
        # keep only the gass columns
        old_data = old_data[gas_cols]
    
    new_data = pd.read_csv(a4th_folder + station_id + '.csv',na_values='-').set_index('datetime')
    new_data.columns = [s.split(' (')[0] for s in new_data.columns]
    # keep only the gass columns
    new_data = new_data[gas_cols]
    # concatinate data 
    data = pd.concat([old_data,new_data])
    filename = a4th_folder_p+station_id + '.csv'
    print(filename)
    data.to_csv(filename)

../data/a4th_hourly/35t.csv
../data/a4th_hourly/36t.csv
../data/a4th_hourly/m9.csv
../data/a4th_hourly/o10.csv
../data/a4th_hourly/o20.csv
../data/a4th_hourly/o22.csv


In [132]:
# parse station informatoin
re = requests.get('https://www.cmuccdc.org/download?fbclid=IwAR0XaoY1G5BSwHcQejEdGD5piw3LSLK4HLlmy4ElzIEBCmVkH2W6T8mTIx8')
soup = BeautifulSoup(re.text)
results = soup.find_all('li', class_='mb-3')

# find chiangmai stations
for item in results:
    if 'เชียงใหม่' in item.text:
        cmu_soup = item
        break
cmu_dustboy_ids = []
for cmu_html in cmu_soup.find_all('a'):
    href = cmu_html['href']
    if 'json' in href:
        dustboy_id = href.split('/')[-1]
        cmu_dustboy_ids.append(dustboy_id)
        
print(cmu_dustboy_ids)
print(len(cmu_dustboy_ids))

['108', '12', '2', '11', '5', '103', '5267', '109', '5264', '2004', '5281', '1', '83', '84', '4008', '4009', '4010', '4011', '4012', '4003', '4004', '4005', '4006', '4007', '6003', '2006', '5263', '20', '106', '4015', '4016', '4002', '6', '5163', '22', '27', '29', '38', '14', '89', '15', '32', '25', '23', '5376', '17', '19', '37', '24', '30', '16', '31', '34', '26', '18', '13', '35', '80', '78', '77', '79', '81', '41', '92', '5374', '5373', '5249', '5262', '5379', '2005', '28', '33', '36', '3', '2002', '5268', '7', '82', '5265', '5266', '5270', '5269', '5248', '6002', '4026', '4027', '4']
87


87 stations at different locations

In [152]:
# cmucdc data 
with open(cdc_folder+'station_info.json', 'r') as f:
    cdc_stations = json.load(f)

cmucdc_stations = []
# search for station information
for station in cdc_stations:
    if station['dustboy_id'] in cmu_dustboy_ids:
        
        try:
            df = pd.read_csv(cdc_folder + station['dustboy_id'] + '.csv')
        except:
            pass
        else:
            # keep only good station with data 
            cmucdc_stations.append(station)
        
len(cmucdc_stations)

25

In [153]:
cmucdc_stations

[{'dustboy_id': '5373',
  'dustboy_uri': 'thapaegate2',
  'dustboy_name_th': 'ลานประตูท่าแพ  (2)',
  'dustboy_name_en': '',
  'dustboy_lat': '18.787726',
  'dustboy_lng': '98.993370',
  'dustboy_version': 'mini'},
 {'dustboy_id': '5374',
  'dustboy_uri': 'thapaegate1',
  'dustboy_name_th': 'ลานประตูท่าแพ  (1)',
  'dustboy_name_en': '',
  'dustboy_lat': '18.787716',
  'dustboy_lng': '98.993563',
  'dustboy_version': 'mini'},
 {'dustboy_id': '5262',
  'dustboy_uri': 'N-112',
  'dustboy_name_th': 'สนามกีฬากลาง มหาวิทยาลัยเชียงใหม่',
  'dustboy_name_en': 'The Main Stadium, Chiang Mai University',
  'dustboy_lat': '18.797338',
  'dustboy_lng': '98.956578',
  'dustboy_version': 'mini'},
 {'dustboy_id': '5268',
  'dustboy_uri': 'N-118',
  'dustboy_name_th': 'สำนักบริการวิชาการ  มหาวิทยาลัยเชียงใหม่ ',
  'dustboy_name_en': 'University Academic  Service Center (UNISERV), Chiang Mai University',
  'dustboy_lat': '18.794057',
  'dustboy_lng': '98.966521',
  'dustboy_version': 'mini'},
 {'dustboy_

# Chiangmai Geography

In [172]:
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'